In [51]:
#!pip install yolov5

In [2]:
import cv2
import torch
import numpy as np
from yolov5 import YOLOv5
from torchvision import transforms

In [3]:
# Initialize YOLOv5 model
import cv2
import torch
import numpy as np
from yolov5 import detect  # YOLOv5 detect module

# Load YOLOv5 model using torch hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Transformation pipeline for preprocessing
preprocess = transforms.Compose([
    transforms.ToTensor(),  # Convert frame to tensor
    transforms.Resize((640, 640)),  # Resize to YOLOv5's expected input size
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-6 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
!git clone https://github.com/abewley/sort.git
!cd sort
!pip install -r requirements.txt


fatal: destination path 'sort' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
!pip install filterpy

In [6]:
# sort.py (a basic SORT tracker implementation)

import numpy as np
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter

def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
              (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh)
    return o

class KalmanBoxTracker:
    count = 0
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0],
                              [0,1,0,0,0,1,0],
                              [0,0,1,0,0,0,1],
                              [0,0,0,1,0,0,0],
                              [0,0,0,0,1,0,0],
                              [0,0,0,0,0,1,0],
                              [0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0],
                              [0,1,0,0,0,0,0],
                              [0,0,1,0,0,0,0],
                              [0,0,0,1,0,0,0]])
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01
        self.kf.x[:4] = self.convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(self.convert_bbox_to_z(bbox))

    def predict(self):
        if (self.kf.x[6]+self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.convert_x_to_bbox(self.kf.x))
        return self.history[-1]

    def get_state(self):
        return self.convert_x_to_bbox(self.kf.x)

    def convert_bbox_to_z(self, bbox):
        w = bbox[2] - bbox[0]
        h = bbox[3] - bbox[1]
        x = bbox[0] + w/2.
        y = bbox[1] + h/2.
        s = w * h
        r = w / float(h)
        return np.array([x, y, s, r]).reshape((4,1))

    def convert_x_to_bbox(self, x):
        w = np.sqrt(x[2] * x[3])
        h = x[2] / w
        return np.array([x[0] - w / 2., x[1] - h / 2., x[0] + w / 2., x[1] + h / 2.]).reshape((1,4))

class Sort:
    def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets):
        self.frame_count += 1
        trks = np.zeros((len(self.trackers), 5))
        to_del = []
        ret = []
        for t, trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets, trks, self.iou_threshold)
        for t, trk in enumerate(self.trackers):
            if t not in unmatched_trks:
                trk.update(dets[matched[np.where(matched[:, 1] == t)[0], 0], :][0])
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i,:])
            self.trackers.append(trk)
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and (trk.hits >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((d, [trk.id+1])).reshape(1, -1))
            i -= 1
            if trk.time_since_update > self.max_age:
                self.trackers.pop(i)
        if len(ret) > 0:
            return np.concatenate(ret)
        return np.empty((0, 5))

def associate_detections_to_trackers(detections, trackers, iou_threshold=0.3):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0,5), dtype=int)
    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d, t] = iou(det, trk)
    matched_indices = linear_sum_assignment(-iou_matrix)
    matched_indices = np.array(list(zip(matched_indices[0], matched_indices[1])))
    unmatched_detections = []
    for d, det in enumerate(detections):
        if d not in matched_indices[:,0]:
            unmatched_detections.append(d)
    unmatched_trackers = []
    for t, trk in enumerate(trackers):
        if t not in matched_indices[:,1]:
            unmatched_trackers.append(t)
    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


In [12]:
import matplotlib.pyplot as plt

# Specify the path to the existing video file
video_path = '/content/traffic_video_short_1.mp4'  # Update this path

# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video loaded successfully.")

# Extract frames and display them
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Convert BGR (OpenCV) to RGB (for displaying with Matplotlib)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # # Display the frame
    # plt.figure(figsize=(10, 6))
    # plt.imshow(frame_rgb)
    # plt.title(f"Frame {frame_count}")
    # plt.axis('off')
    # plt.show()

cap.release()
print(f"Total frames extracted: {frame_count}")


Video loaded successfully.
Total frames extracted: 316


In [13]:
import cv2
import torch
import numpy as np
#from sort import Sort  # Import the SORT implementation

# Initialize YOLOv5 model from PyTorch hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Initialize SORT tracker
tracker = Sort()

video_path = '/content/traffic_video_short_1.mp4'  # Update this path

# Load video (use a video from Google Colab or your Drive)
# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video loaded successfully.")

# Prepare to write the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
output_video_path = 'output.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

# Define the list of classes you want to track (in this case, 'car', 'truck', 'bus', etc.)
VEHICLE_CLASSES = [2, 3, 5, 7]  # 2: car, 3: motorcycle, 5: bus, 7: truck

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"Processing finished at frame {frame_count}.")
        break

    frame_count += 1

    # Convert BGR (OpenCV) to RGB (YOLOv5 expects RGB format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform vehicle detection using YOLOv5
    results = model(frame_rgb)

    # Extract detections: YOLOv5 gives us a list of detected objects
    detections = results.xyxy[0].cpu().numpy()  # Get the detections as a NumPy array

    # Prepare bounding boxes for SORT (xmin, ymin, xmax, ymax, confidence, class)
    vehicle_boxes = []
    for *xyxy, conf, cls in detections:
        if int(cls) in VEHICLE_CLASSES:  # Filter by vehicle classes
            vehicle_boxes.append([*xyxy, conf])  # Bounding box and confidence only for SORT

    # Convert vehicle_boxes to NumPy array for SORT
    if len(vehicle_boxes) > 0:
        vehicle_boxes = np.array(vehicle_boxes)

        # Update the tracker with the current frame's detections
        tracked_objects = tracker.update(vehicle_boxes)

        # Draw the tracked bounding boxes on the frame
        for obj in tracked_objects:
            x1, y1, x2, y2, track_id = obj
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw the bounding box and the tracking ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {int(track_id)}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f"Output video saved as {output_video_path}.")

# Display a sample frame from the video to confirm processing
cap = cv2.VideoCapture(output_video_path)
# if cap.isOpened():
#     ret, frame = cap.read()
#     if ret:
#         plt.figure(figsize=(10, 6))
#         plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#         plt.title("Sample Frame from Output Video")
#         plt.axis('off')
#         plt.show()
#     cap.release()

# Download the output video
from google.colab import files
files.download(output_video_path)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-6 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Video loaded successfully.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

Processing finished at frame 316.
Output video saved as output.mp4.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
video_path = '/content/traffic_video_short_1.mp4'  # Update this path

# Open the video file
# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video loaded successfully.")

# Prepare to write the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
output_video_path = 'output.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

# Define the list of classes you want to track (in this case, 'car', 'truck', 'bus', etc.)
VEHICLE_CLASSES = {2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck'}  # Class ID to Name
#vehicle_tracker_ids = {vehicle: set() for vehicle in VEHICLE_CLASSES.values()}

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"Processing finished at frame {frame_count}.")
        break

    frame_count += 1

    # Convert BGR (OpenCV) to RGB (YOLOv5 expects RGB format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform vehicle detection using YOLOv5
    results = model(frame_rgb)

    # Extract detections: YOLOv5 gives us a list of detected objects
    detections = results.xyxy[0].cpu().numpy()  # Get the detections as a NumPy array

    # Initialize counts and tracking data
    tracked_vehicle_ids = {vehicle: [] for vehicle in VEHICLE_CLASSES.keys()}

    vehicle_boxes=[]
    for *xyxy, conf, cls in detections:
        cls = cls
        # Append only boxes for tracking (without class info for SORT)
        if cls in VEHICLE_CLASSES.keys():
          vehicle_boxes.append([*xyxy, conf])
          x1, y1, x2, y2, track_id = *xyxy,conf
          tracked_vehicle_ids[cls].append(track_id)
        #print(f"Object ID: {int(track_id)}, BBox: ({int(x1)}, {int(y1)}, {int(x2)}, {int(y2)})")

    # Convert vehicle_boxes to NumPy array for SORT
    if len(vehicle_boxes) > 0:
        vehicle_boxes = np.array(vehicle_boxes)

        # Update the tracker with the current frame's detections
        tracked_objects = tracker.update(vehicle_boxes)

        # Print IDs of objects in the current frame
        #print(f"Frame {frame_count} object IDs:")

        # Draw the tracked bounding boxes on the frame
        for obj in tracked_objects:
            x1, y1, x2, y2, track_id = obj
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw the bounding box and the tracking ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {int(track_id)}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    #print(tracked_vehicle_ids)
    for v,k in zip(tracked_vehicle_ids.keys(),tracked_vehicle_ids.values()):
      print('Class',VEHICLE_CLASSES[v],"count",len(k))
    #print(tracked_vehicle_ids)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f"Output video saved as {output_video_path}.")

# Print vehicle counts
print("Vehicle counts:")
for vehicle, count in vehicle_counts.items():
    print(f"{vehicle}: {count}")

# Download the output video
from google.colab import files
files.download(output_video_path)

Video loaded successfully.
Class car count 19
Class motorcycle count 0
Class bus count 1
Class truck count 3


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 18
Class motorcycle count 0
Class bus count 2
Class truck count 3


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 19
Class motorcycle count 0
Class bus count 1
Class truck count 3


...

Class car count 19
Class motorcycle count 0
Class bus count 0
Class truck count 1


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 19
Class motorcycle count 0
Class bus count 0
Class truck count 1
Processing finished at frame 316.
Output video saved as output.mp4.
Vehicle counts:
car: 0
motorcycle: 0
bus: 0
truck: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
video_path = '/content/traffic_video_short_1.mp4'  # Update this path

# Open the video file
# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video loaded successfully.")

# Prepare to write the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
output_video_path = 'output.mp4'
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (frame_width, frame_height))

# Define the list of classes you want to track (in this case, 'car', 'truck', 'bus', etc.)
VEHICLE_CLASSES = {2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck'}  # Class ID to Name
#vehicle_tracker_ids = {vehicle: set() for vehicle in VEHICLE_CLASSES.values()}

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print(f"Processing finished at frame {frame_count}.")
        break

    frame_count += 1

    # Convert BGR (OpenCV) to RGB (YOLOv5 expects RGB format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform vehicle detection using YOLOv5
    results = model(frame_rgb)

    # Extract detections: YOLOv5 gives us a list of detected objects
    detections = results.xyxy[0].cpu().numpy()  # Get the detections as a NumPy array

    # Initialize counts and tracking data
    tracked_vehicle_ids = {vehicle: [] for vehicle in VEHICLE_CLASSES.keys()}

    vehicle_boxes=[]
    for *xyxy, conf, cls in detections:
        cls = cls
        # Append only boxes for tracking (without class info for SORT)
        if cls in VEHICLE_CLASSES.keys():
          vehicle_boxes.append([*xyxy, conf])
          x1, y1, x2, y2, track_id = *xyxy,conf
          tracked_vehicle_ids[cls].append(track_id)
        #print(f"Object ID: {int(track_id)}, BBox: ({int(x1)}, {int(y1)}, {int(x2)}, {int(y2)})")

    # Convert vehicle_boxes to NumPy array for SORT
    if len(vehicle_boxes) > 0:
        vehicle_boxes = np.array(vehicle_boxes)

        # Update the tracker with the current frame's detections
        tracked_objects = tracker.update(vehicle_boxes)

        # Print IDs of objects in the current frame
        #print(f"Frame {frame_count} object IDs:")

        # Draw the tracked bounding boxes on the frame
        for obj in tracked_objects:
            x1, y1, x2, y2, track_id = obj
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw the bounding box and the tracking ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {int(track_id)}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    #print(tracked_vehicle_ids)
    for v,k in zip(tracked_vehicle_ids.keys(),tracked_vehicle_ids.values()):
      print('Class',VEHICLE_CLASSES[v],"count",len(k))
    #print(tracked_vehicle_ids)

    y_offset = 30
    for vehicle_class, count in tracked_vehicle_ids.items():
        cv2.putText(frame, f"{VEHICLE_CLASSES[vehicle_class]}: {count}", (10, y_offset),1, cv2.FONT_HERSHEY_SIMPLEX,(0, 255, 0), 2)
        y_offset += 30

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f"Output video saved as {output_video_path}.")

# Print vehicle counts
print("Vehicle counts:")
for vehicle, count in vehicle_counts.items():
    print(f"{vehicle}: {count}")

# Download the output video
from google.colab import files
files.download(output_video_path)

Video loaded successfully.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 19
Class motorcycle count 0
Class bus count 1
Class truck count 3


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 18
Class motorcycle count 0
Class bus count 2
Class truck count 3


...

Class car count 19
Class motorcycle count 0
Class bus count 0
Class truck count 1


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Class car count 19
Class motorcycle count 0
Class bus count 0
Class truck count 1
Processing finished at frame 316.
Output video saved as output.mp4.
Vehicle counts:
car: 0
motorcycle: 0
bus: 0
truck: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>